In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import string
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import roc_auc_score
from torch.optim import Adam, lr_scheduler
from sklearn.model_selection import train_test_split

In [2]:
def prep(series):
    for c in 'ABTCE':  
        series[c+'_count'] = series['f_27'].str.count(c)
    for i in range(10):
        series['pos'+str(i)] = series['f_27'].str.get(i).apply(ord) - ord('A')
    series['unique'] = series['f_27'].apply(lambda s:len(set(s)))
    series['i_02_21'] = (series.f_21 + series.f_02 > 5.2).astype(int) - (series.f_21 + series.f_02 < -5.3).astype(int)
    series['i_05_22'] = (series.f_22 + series.f_05 > 5.1).astype(int) - (series.f_22 + series.f_05 < -5.4).astype(int)
    i_00_01_26 = series.f_00 + series.f_01 + series.f_26
    series['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)
    series['f_28'] = series['f_28']/100
    return series.drop(columns=['f_27'])

In [3]:
train_series = prep(pd.read_csv('train.csv'))
valid_series = prep(pd.read_csv('test.csv'))

In [4]:
X = train_series.drop(columns=['id','target']).values
y = train_series['target'].values
X_valid = valid_series.drop(columns=['id']).values

In [5]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.1, random_state=123)

In [6]:
XX = np.vstack((X_train,X_holdout,X_valid))

In [7]:
sc = StandardScaler()
sc.fit(XX)

StandardScaler()

In [8]:
sc.mean_, sc.scale_

(array([-1.99139759e-04,  3.35231009e-04,  4.91865108e-04, -4.61139352e-04,
         2.90640663e-05, -4.48457907e-05,  7.16404397e-04,  2.03117937e+00,
         2.05752750e+00,  2.36188875e+00,  2.17737500e+00,  1.80438312e+00,
         2.84253375e+00,  2.23923062e+00,  1.51513937e+00,  2.10080687e+00,
         2.09548500e+00,  1.85842750e+00,  2.06643625e+00,  3.08084317e-01,
        -1.76648401e-01, -1.55270565e-01, -7.67616258e-03, -3.70562894e-01,
        -3.41839023e-01,  1.75540783e-01,  3.57020587e-01,  5.01000667e-04,
         3.45848750e-01,  1.00284125e+00,  2.46214750e+00,  3.24328125e+00,
         4.34975000e-02,  1.47834812e+00,  5.97812500e-01,  4.98809375e-01,
         1.96664375e+00,  6.68870000e-01,  1.99077938e+00,  1.38585250e+00,
         4.67763125e-01,  2.37539938e+00,  9.36298312e+00,  2.02507750e+00,
         2.28332312e+00,  5.01429250e+00, -4.62937500e-03,  2.48875000e-03,
         1.68556250e-02]),
 array([0.99948944, 0.99829086, 1.00042761, 1.00086758, 0.999

In [9]:
X_train = sc.transform(X_train)
X_holdout = sc.transform(X_holdout)
X_valid = sc.transform(X_valid)

In [10]:
X_train[:2]

array([[ 0.06604802, -1.95217896,  2.34234642, -0.50997358, -0.19980724,
         0.55424035, -1.55428309,  0.58501675,  2.47753031,  0.38974266,
         1.10759162, -1.17315464, -1.04573086, -0.15556781, -0.37861093,
         0.57308   ,  2.50232099, -1.26593251, -0.6816248 , -1.03067016,
        -1.00835187,  0.94427169,  0.85987451,  0.31770311,  2.05069675,
         0.46456005, -0.52402869, -0.91150446,  1.37529538, -0.00346795,
        -1.08415031, -0.17087815, -0.21325003,  0.49031975,  1.90874495,
         1.00238409,  0.02224128,  0.70360417,  1.2586021 , -1.06741292,
         1.06669311,  1.5858311 , -1.29050608,  1.2326551 ,  1.6434655 ,
        -0.01602634,  0.0205493 , -0.01142337, -0.05907975],
       [-2.23754584,  1.02617583, -0.181566  , -0.46183511, -0.13567033,
         0.6929298 , -1.29213315, -0.01882749,  2.47753031, -0.83180815,
        -0.10778908, -0.52298527,  0.08937004, -1.45613551, -1.11357887,
         1.84773387,  0.57968451, -0.58474774,  0.59669784,  0.

In [11]:
np.std(X_train[:,0])

0.9994984181282446

In [12]:
class MyModel(nn.Module):
    def __init__(self, in_channels=49):
        super().__init__()
        self.fc1 = nn.Linear(in_channels, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64,32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32,1)
        self.dropout = nn.Dropout(p=0.02)
    
    def forward(self,X):
        X = self.bn1(F.relu(self.fc1(X)))
        X = self.dropout(X)
        X = self.bn2(F.relu(self.fc2(X)))
        X = self.dropout(X)
        X = self.bn3(F.relu(self.fc3(X)))
        X = self.dropout(X)
        X = self.fc4(X)
        X = torch.sigmoid(X).squeeze(dim=-1)
        return X

In [13]:
class MyDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = X
        if y is None:
            self.y = None
        else:
            self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        ''' input tensor shape is N*C '''
        X = self.X[idx]
        if self.y is None:
            y = -1
        else:
            y = self.y[idx]
        return (torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32))

In [61]:
def evaluate(model, test_loader):
    model.eval()
    with torch.no_grad():
        X, y = next(iter(test_loader))
        output = model(X)
        print(output[:10], y[:10])
        score = roc_auc_score(y.detach().numpy(), output.detach().numpy())
        loss = nn.BCELoss()(output, y)
    return score, loss

In [62]:
del model

In [63]:
0.98**200

0.0175879466057215

In [64]:
def train(Xdata, ydata, epoches=200, batch_size=1024):
    torch.manual_seed(1)
    for isplit, (train_idx, test_idx) in enumerate(kf.split(Xdata)):
        
        model = MyModel()
        
        X_train, y_train = Xdata[train_idx], ydata[train_idx]
        X_test, y_test = Xdata[test_idx], ydata[test_idx]
        train_dataset = MyDataset(X_train, y_train)
        test_dataset = MyDataset(X_test, y_test)
        train_loader = DataLoader(train_dataset, shuffle=True, drop_last=False, batch_size=batch_size)
        test_loader = DataLoader(test_dataset, shuffle=False, drop_last=False, batch_size=len(X_test))
    
        criteria = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
        lr_decay = lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

        best_score = 0.0
        for epoch in range(epoches):

            curr_loss = 0.0
            model.train()

            for batch, (X, y) in enumerate(train_loader):
                output = model(X)            
                loss = criteria(output, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                curr_loss += loss.item() * len(y)

            curr_loss /= len(train_loader.dataset)
            score, test_loss = evaluate(model, test_loader)
            print(f"{epoch}: {curr_loss}; roc={score} test_loss={test_loss}")
            lr_decay.step()
            if (score > best_score):
                torch.save({'train_loss':curr_loss/len(train_loader.dataset),
                    'test_loss': test_loss,
                    'score': score,
                    'model': model.state_dict()},'nn_model_'+str(isplit)+'.pickle')

In [65]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [57]:
X_train.shape, y_train.shape, X_holdout.shape, y_holdout.shape, X_valid.shape

((810000, 49), (810000,), (90000, 49), (90000,), (700000, 49))

In [66]:
train(X_train, y_train, batch_size=256*10, epoches=200)

tensor([0.9849, 0.3183, 0.0011, 0.9986, 0.2988, 0.9787, 0.9989, 0.7914, 0.9119,
        0.4402]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
0: 0.2414736752892718; roc=0.9860537404519322 test_loss=0.1509825736284256
tensor([9.7695e-01, 1.0298e-01, 2.2160e-04, 9.9733e-01, 1.5074e-01, 9.8812e-01,
        9.9943e-01, 9.7539e-01, 9.8597e-01, 5.3505e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
1: 0.14229276565122015; roc=0.9918313703309842 test_loss=0.12037302553653717
tensor([9.9877e-01, 1.5369e-01, 2.8201e-04, 9.9919e-01, 7.2781e-01, 9.9976e-01,
        9.9995e-01, 9.8754e-01, 9.9880e-01, 6.1525e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
2: 0.11978810336854723; roc=0.9930340349245599 test_loss=0.11056447774171829
tensor([9.9628e-01, 2.0098e-02, 4.5505e-04, 9.9953e-01, 2.0080e-01, 9.9980e-01,
        9.9974e-01, 9.3796e-01, 9.4645e-01, 5.9536e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
3: 0.10959612123760176; roc=0.9940295436413826 test_loss=0.11936135

tensor([9.9948e-01, 4.0310e-03, 4.3275e-07, 9.9950e-01, 3.8228e-01, 9.9999e-01,
        9.9998e-01, 9.9819e-01, 9.9988e-01, 9.6398e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
32: 0.0691382583865413; roc=0.9973638892218732 test_loss=0.06503008306026459
tensor([9.9982e-01, 1.6821e-03, 7.1773e-07, 9.9955e-01, 5.2179e-01, 9.9999e-01,
        9.9999e-01, 9.9911e-01, 9.9968e-01, 9.4895e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
33: 0.06917070907391148; roc=0.9972756410650498 test_loss=0.06611042469739914
tensor([9.9988e-01, 7.6493e-03, 7.9672e-07, 9.9927e-01, 2.2060e-01, 1.0000e+00,
        9.9999e-01, 9.9662e-01, 9.9938e-01, 9.7017e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
34: 0.0686960929117085; roc=0.9974223105094353 test_loss=0.06445206701755524
tensor([9.9987e-01, 4.5938e-03, 2.0264e-07, 9.9981e-01, 3.0854e-01, 9.9994e-01,
        1.0000e+00, 9.9912e-01, 9.9992e-01, 9.9364e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
35: 0.06884475665695873;

tensor([9.9998e-01, 1.7801e-03, 3.2660e-07, 9.9986e-01, 3.6011e-01, 1.0000e+00,
        1.0000e+00, 9.9943e-01, 9.9963e-01, 9.6387e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
64: 0.06297776126199299; roc=0.9976395998210159 test_loss=0.06138982996344566
tensor([9.9997e-01, 3.5032e-03, 3.4719e-07, 9.9988e-01, 4.0014e-01, 1.0000e+00,
        1.0000e+00, 9.9876e-01, 9.9906e-01, 9.8133e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
65: 0.06308538401568377; roc=0.9976365635432962 test_loss=0.0614449642598629
tensor([9.9999e-01, 5.4448e-03, 4.5507e-07, 9.9989e-01, 2.1160e-01, 1.0000e+00,
        1.0000e+00, 9.9893e-01, 9.9964e-01, 9.6550e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
66: 0.06299244400159812; roc=0.9976324109813942 test_loss=0.06148916110396385
tensor([9.9997e-01, 1.1287e-02, 1.7930e-07, 9.9994e-01, 3.5075e-01, 1.0000e+00,
        1.0000e+00, 9.9984e-01, 9.9982e-01, 9.8951e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
67: 0.06287299485486231

tensor([9.9998e-01, 1.7273e-03, 2.3891e-07, 9.9995e-01, 5.4104e-01, 1.0000e+00,
        1.0000e+00, 9.9899e-01, 9.9958e-01, 9.7341e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
96: 0.060499721073809964; roc=0.9976572689940277 test_loss=0.061910808086395264
tensor([9.9999e-01, 3.6566e-03, 9.9468e-08, 9.9997e-01, 3.3435e-01, 1.0000e+00,
        1.0000e+00, 9.9923e-01, 9.9970e-01, 9.6501e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
97: 0.0601497453984655; roc=0.9976649502648541 test_loss=0.061901066452264786
tensor([1.0000e+00, 1.9630e-03, 2.6172e-07, 9.9994e-01, 5.3445e-01, 1.0000e+00,
        1.0000e+00, 9.9953e-01, 9.9945e-01, 9.8626e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
98: 0.06031804435415032; roc=0.9976904171805709 test_loss=0.06099728122353554
tensor([1.0000e+00, 2.3286e-03, 7.3919e-08, 9.9997e-01, 4.4942e-01, 1.0000e+00,
        1.0000e+00, 9.9943e-01, 9.9973e-01, 9.7473e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
99: 0.06020815771670

tensor([1.0000e+00, 2.8198e-03, 7.6051e-08, 9.9997e-01, 5.0723e-01, 1.0000e+00,
        1.0000e+00, 9.9918e-01, 9.9975e-01, 9.8706e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
127: 0.058988071939459555; roc=0.997714087668201 test_loss=0.06097738817334175
tensor([1.0000e+00, 2.1937e-03, 1.1306e-07, 9.9996e-01, 6.5472e-01, 1.0000e+00,
        1.0000e+00, 9.9949e-01, 9.9976e-01, 9.8814e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
128: 0.05883639650580324; roc=0.9976978151661597 test_loss=0.06066720932722092
tensor([1.0000e+00, 3.2356e-03, 1.2303e-07, 9.9996e-01, 5.4892e-01, 1.0000e+00,
        1.0000e+00, 9.9911e-01, 9.9960e-01, 9.8672e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
129: 0.058699520804263924; roc=0.9976986462582298 test_loss=0.060833923518657684
tensor([1.0000e+00, 4.4144e-03, 1.2924e-07, 9.9996e-01, 6.2778e-01, 1.0000e+00,
        1.0000e+00, 9.9969e-01, 9.9983e-01, 9.8944e-01]) tensor([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.])
130: 0.0588913581

...

tensor([3.3137e-04, 1.1068e-05, 6.7794e-03, 8.2212e-01, 2.1966e-03, 2.1763e-02,
        9.9988e-01, 8.7059e-01, 9.9992e-01, 9.9983e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
61: 0.06388836925780332; roc=0.9974592944019829 test_loss=0.06538308411836624
tensor([1.1334e-04, 4.0367e-07, 4.5040e-03, 5.4337e-01, 8.7617e-04, 1.8768e-03,
        9.9995e-01, 7.3615e-01, 9.9995e-01, 9.9984e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
62: 0.06330342657588146; roc=0.9974650492562783 test_loss=0.06423264741897583
tensor([7.3679e-05, 6.0750e-06, 3.3983e-03, 7.2829e-01, 3.4712e-04, 1.6299e-03,
        9.9993e-01, 8.9406e-01, 9.9997e-01, 9.9976e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
63: 0.06302928463912305; roc=0.9974643348279639 test_loss=0.0639273151755333
tensor([2.6958e-04, 4.3251e-06, 5.5585e-03, 8.1162e-01, 9.8329e-04, 1.0462e-02,
        9.9999e-01, 8.6045e-01, 9.9994e-01, 9.9989e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
64: 0.0630089260858518;

tensor([6.9856e-05, 3.7917e-06, 4.4134e-03, 6.6821e-01, 3.5539e-04, 2.2249e-03,
        9.9997e-01, 8.6497e-01, 9.9993e-01, 9.9925e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
93: 0.060716319801630794; roc=0.997549146345866 test_loss=0.06281903386116028
tensor([9.7610e-05, 1.1825e-06, 8.6874e-03, 7.3564e-01, 2.6311e-04, 4.0534e-03,
        9.9997e-01, 8.8731e-01, 9.9996e-01, 9.9938e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
94: 0.06053753976100757; roc=0.9975466691831302 test_loss=0.06292284280061722
tensor([1.5483e-04, 6.5208e-07, 4.1030e-03, 6.1039e-01, 1.6023e-04, 1.6359e-03,
        9.9997e-01, 8.4592e-01, 9.9997e-01, 9.9971e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
95: 0.06049763017230564; roc=0.9975620293156294 test_loss=0.06284051388502121
tensor([1.4473e-04, 7.6545e-06, 5.5489e-03, 7.0293e-01, 3.1447e-04, 4.2680e-03,
        9.9992e-01, 8.5214e-01, 9.9994e-01, 9.9962e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
96: 0.0603312394887576

tensor([1.0190e-04, 7.7596e-06, 5.1825e-03, 7.2669e-01, 2.0574e-04, 2.0351e-03,
        9.9996e-01, 9.0185e-01, 9.9999e-01, 9.9974e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
124: 0.05890515970963019; roc=0.997586952781886 test_loss=0.06230543926358223
tensor([1.2218e-04, 3.1540e-06, 6.8304e-03, 6.8141e-01, 1.7082e-04, 2.7159e-03,
        9.9996e-01, 9.0809e-01, 9.9998e-01, 9.9967e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
125: 0.05892089132173562; roc=0.9975738664196896 test_loss=0.062477901577949524
tensor([8.1447e-05, 1.7876e-06, 5.5598e-03, 7.3502e-01, 1.9906e-04, 2.4065e-03,
        9.9997e-01, 9.0543e-01, 9.9998e-01, 9.9975e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
126: 0.05906649897495905; roc=0.9975753656667257 test_loss=0.06249835342168808
tensor([8.3279e-05, 4.4750e-06, 4.8656e-03, 6.9667e-01, 1.6968e-04, 1.4260e-03,
        9.9998e-01, 8.7637e-01, 9.9999e-01, 9.9978e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
127: 0.058545288318

tensor([5.7036e-05, 1.6559e-06, 5.8115e-03, 7.0534e-01, 1.4257e-04, 1.9428e-03,
        9.9997e-01, 9.2430e-01, 9.9998e-01, 9.9951e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
155: 0.05838200270393748; roc=0.9975904989179019 test_loss=0.062411047518253326
tensor([8.6374e-05, 2.0305e-06, 5.5148e-03, 6.0025e-01, 1.3921e-04, 1.7636e-03,
        9.9996e-01, 8.7597e-01, 9.9998e-01, 9.9953e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
156: 0.058503416513587224; roc=0.9975950209105268 test_loss=0.062230877578258514
tensor([6.6796e-05, 3.3989e-06, 4.4661e-03, 6.6819e-01, 1.6617e-04, 1.9355e-03,
        9.9998e-01, 8.7752e-01, 9.9999e-01, 9.9951e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
157: 0.05818524726379065; roc=0.997596459833823 test_loss=0.062260646373033524
tensor([6.0999e-05, 2.4582e-06, 5.1232e-03, 6.6239e-01, 1.0622e-04, 2.1219e-03,
        9.9995e-01, 9.1161e-01, 9.9998e-01, 9.9931e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
158: 0.057947397

tensor([7.8635e-05, 1.6125e-06, 5.0702e-03, 6.4060e-01, 9.8020e-05, 2.2556e-03,
        9.9998e-01, 9.0114e-01, 9.9999e-01, 9.9973e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
186: 0.05771808055815873; roc=0.9975940067700775 test_loss=0.06234874203801155
tensor([5.7666e-05, 2.0667e-06, 5.6100e-03, 6.3582e-01, 1.0242e-04, 2.1890e-03,
        9.9997e-01, 9.0875e-01, 9.9998e-01, 9.9958e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
187: 0.05772979278991252; roc=0.9975915545452214 test_loss=0.06231987848877907
tensor([7.2969e-05, 1.4216e-06, 5.5446e-03, 6.6931e-01, 8.8791e-05, 1.8989e-03,
        9.9997e-01, 8.9995e-01, 9.9999e-01, 9.9971e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
188: 0.05769781435713356; roc=0.9975881526218382 test_loss=0.06242556497454643
tensor([5.9073e-05, 2.3220e-06, 4.6984e-03, 6.5397e-01, 1.2210e-04, 1.7602e-03,
        9.9997e-01, 9.0926e-01, 9.9998e-01, 9.9962e-01]) tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])
189: 0.057955646146

In [67]:
pd_holdout = pd.DataFrame()
pd_holdout['target'] = y_holdout
for isplit in range(5):
    model = MyModel()
    model.load_state_dict(torch.load(f'nn_model_{isplit}.pickle')['model'])
    model.eval()
    with torch.no_grad():
        pd_holdout[f'{isplit}'] = model(torch.Tensor(X_holdout))

In [69]:
pd_holdout['nn_prob'] = pd_holdout[['0','1','2','3','4']].mean(axis=1)
pd_holdout

,target,0,1,2,3,4,nn_prob
0,1,9.999733e-01,0.999955,9.996197e-01,9.998216e-01,9.999269e-01,9.998593e-01
1,0,3.482655e-07,0.000002,6.849074e-09,7.695823e-08,3.159556e-07,5.754856e-07
2,1,9.999990e-01,0.999993,9.999303e-01,9.999985e-01,9.999793e-01,9.999800e-01
3,1,9.654357e-01,0.728226,9.791028e-01,9.452514e-01,7.914397e-01,8.818913e-01
4,1,9.996759e-01,0.999997,9.999877e-01,9.998951e-01,9.999548e-01,9.999021e-01
...,...,...,...,...,...,...,...
89995,0,3.189360e-03,0.010664,9.675940e-03,1.423564e-02,2.188519e-02,1.192996e-02
89996,1,9.927418e-01,0.999911,9.983478e-01,9.996113e-01,9.990080e-01,9.979240e-01
89997,0,9.094466e-08,0.000002,1.584435e-07,1.880491e-07,1.264559e-05,2.957890e-06
89998,1,3.431977e-02,0.037310,5.278426e-02,4.051618e-02,9.134308e-02,5.125475e-02


In [70]:
tmp = pd.read_csv('xgboost_holdout.csv')
pd_holdout['xgboost_prob'] = tmp['xgboost_prob']
pd_holdout

,target,0,1,2,3,4,nn_prob,xgboost_prob
0,1,9.999733e-01,0.999955,9.996197e-01,9.998216e-01,9.999269e-01,9.998593e-01,9.991639e-01
1,0,3.482655e-07,0.000002,6.849074e-09,7.695823e-08,3.159556e-07,5.754856e-07,6.165386e-12
2,1,9.999990e-01,0.999993,9.999303e-01,9.999985e-01,9.999793e-01,9.999800e-01,9.995431e-01
3,1,9.654357e-01,0.728226,9.791028e-01,9.452514e-01,7.914397e-01,8.818913e-01,8.471072e-01
4,1,9.996759e-01,0.999997,9.999877e-01,9.998951e-01,9.999548e-01,9.999021e-01,9.972556e-01
...,...,...,...,...,...,...,...,...
89995,0,3.189360e-03,0.010664,9.675940e-03,1.423564e-02,2.188519e-02,1.192996e-02,6.157296e-02
89996,1,9.927418e-01,0.999911,9.983478e-01,9.996113e-01,9.990080e-01,9.979240e-01,9.982706e-01
89997,0,9.094466e-08,0.000002,1.584435e-07,1.880491e-07,1.264559e-05,2.957890e-06,5.030987e-06
89998,1,3.431977e-02,0.037310,5.278426e-02,4.051618e-02,9.134308e-02,5.125475e-02,9.341167e-03


In [72]:
from sklearn.linear_model import LogisticRegression

In [82]:
xt_holdout, yt_holdout = pd_holdout[['nn_prob','xgboost_prob']].values, pd_holdout['target']
for C in [1e-3,1.0,1e+3,1e+6]:
    results = []
    for train_idx, test_idx in kf.split(xt_holdout):
        xtrain, ytrain = xt_holdout[train_idx], yt_holdout[train_idx]
        xtest, ytest = xt_holdout[test_idx], yt_holdout[test_idx]
        clf = LogisticRegression(C=C)
        clf.fit(xtrain, ytrain)
        print(clf.coef_)
        score = roc_auc_score(ytest, clf.predict_proba(xtest)[:,1])
        results.append(score)
    print(f"{C} {np.array(results).mean(),np.array(results).std()}")

[[2.5990119  2.39813972]]
[[2.59502031 2.39739433]]
[[2.59944025 2.39678854]]
[[2.60256142 2.39113569]]
[[2.59455286 2.38878242]]
0.001 (0.9976131446464184, 0.00029577501018954405)
[[7.02735628 2.89444176]]
[[6.93206938 2.94160468]]
[[7.11560864 2.77906583]]
[[7.20743854 2.66179823]]
[[7.04752643 2.69518673]]
1.0 (0.9977848267006294, 0.00028120909497533975)
[[7.14958079 2.80626739]]
[[7.04847029 2.85828375]]
[[7.24659983 2.68165402]]
[[7.34384684 2.55843532]]
[[7.17545548 2.59768665]]
1000.0 (0.9977916509597968, 0.0002800772788277845)
[[7.14970989 2.80617272]]
[[7.04859307 2.8581944 ]]
[[7.24673853 2.68154929]]
[[7.3439914  2.55832419]]
[[7.17559088 2.59758197]]
1000000.0 (0.9977916657837987, 0.0002800647367750917)


In [74]:
clf = LogisticRegression(C=1000)
clf.fit(xt_holdout, yt_holdout)

LogisticRegression(C=1000)

### prepare output

In [76]:
pd_valid = pd.DataFrame()
pd_valid['id'] = valid_series['id']

for isplit in range(5):
    model = MyModel()
    model.load_state_dict(torch.load(f'nn_model_{isplit}.pickle')['model'])
    model.eval()
    with torch.no_grad():
        pd_valid[f'{isplit}'] = model(torch.Tensor(X_valid))

In [77]:
pd_valid['nn_prob'] = pd_valid[['0','1','2','3','4']].mean(axis=1)

In [78]:
tmp = pd.read_csv('xgboost_submit_blend.csv')
pd_valid['xgb_prob'] = tmp['target']
pd_valid

,id,0,1,2,3,4,nn_prob,xgb_prob
0,900000,9.999933e-01,9.999940e-01,9.999998e-01,9.999931e-01,9.999944e-01,9.999949e-01,9.999978e-01
1,900001,9.999982e-01,9.999939e-01,9.999727e-01,9.999596e-01,9.999901e-01,9.999828e-01,9.997897e-01
2,900002,4.686103e-07,1.340410e-09,1.200025e-07,1.667394e-09,2.494749e-08,1.233136e-07,1.963870e-11
3,900003,2.266349e-06,1.162745e-03,2.503370e-05,9.388438e-06,3.900878e-05,2.476884e-04,3.375970e-04
4,900004,9.991106e-01,9.981763e-01,9.977717e-01,9.998505e-01,9.986734e-01,9.987165e-01,9.992279e-01
...,...,...,...,...,...,...,...,...
699995,1599995,6.716827e-01,6.506397e-01,7.472442e-01,7.319505e-01,8.485841e-01,7.300202e-01,6.329173e-01
699996,1599996,9.999987e-01,9.999599e-01,9.999831e-01,9.999964e-01,9.999968e-01,9.999870e-01,9.994021e-01
699997,1599997,4.519228e-02,5.872575e-03,8.607415e-02,2.928179e-03,2.247222e-02,3.250788e-02,1.275131e-01
699998,1599998,1.174141e-05,8.198066e-06,6.891507e-07,3.747346e-05,1.518074e-05,1.465657e-05,8.724722e-05


In [79]:
pd_valid['target'] = clf.predict_proba(pd_valid[['nn_prob','xgb_prob']].values)[:,1]

In [80]:
pd_valid

,id,0,1,2,3,4,nn_prob,xgb_prob,target
0,900000,9.999933e-01,9.999940e-01,9.999998e-01,9.999931e-01,9.999944e-01,9.999949e-01,9.999978e-01,0.993705
1,900001,9.999982e-01,9.999939e-01,9.999727e-01,9.999596e-01,9.999901e-01,9.999828e-01,9.997897e-01,0.993701
2,900002,4.686103e-07,1.340410e-09,1.200025e-07,1.667394e-09,2.494749e-08,1.233136e-07,1.963870e-11,0.007922
3,900003,2.266349e-06,1.162745e-03,2.503370e-05,9.388438e-06,3.900878e-05,2.476884e-04,3.375970e-04,0.007943
4,900004,9.991106e-01,9.981763e-01,9.977717e-01,9.998505e-01,9.986734e-01,9.987165e-01,9.992279e-01,0.993635
...,...,...,...,...,...,...,...,...,...
699995,1599995,6.716827e-01,6.506397e-01,7.472442e-01,7.319505e-01,8.485841e-01,7.300202e-01,6.329173e-01,0.893683
699996,1599996,9.999987e-01,9.999599e-01,9.999831e-01,9.999964e-01,9.999968e-01,9.999870e-01,9.994021e-01,0.993695
699997,1599997,4.519228e-02,5.872575e-03,8.607415e-02,2.928179e-03,2.247222e-02,3.250788e-02,1.275131e-01,0.014036
699998,1599998,1.174141e-05,8.198066e-06,6.891507e-07,3.747346e-05,1.518074e-05,1.465657e-05,8.724722e-05,0.007925


In [81]:
pd_valid[['id','target']].to_csv('stack_submit.csv',index=False)